In [ ]:
import nrrd
import np
import matplotlib as plt

In [ ]:
data,header = nrrd.read('annotation_25.nrrd')
print(data.shape)

In [ ]:
volume_indexes = data>0
# write annotation indexes
volume_indexes.astype('uint8').tofile('./compression/volume_indexes.bytes')

In [ ]:
data_flat = data.flatten()
data_flat = data_flat[volume_indexes.flatten()]
uannotations = np.unique(data_flat)
uannotations.astype('uint32').tofile('./compression/annotation_map.bytes')

In [ ]:
annotation_indexes = np.zeros(data_flat.shape)
for i, uann in enumerate(uannotations):
    annotation_indexes[data_flat==uann] = i
annotation_indexes.astype('uint16').tofile('./compression/annotation_indexes.bytes')

In [ ]:
# Re-load all files!
volume_indexes_load = np.fromfile('./compression/volume_indexes.bytes',dtype='uint8')
print(np.all(volume_indexes.flatten()==volume_indexes_load))
annotation_map_load = np.fromfile('./compression/annotation_map.bytes',dtype='uint32')
print(np.all(uannotations==annotation_map_load))
annotation_indexes_load = np.fromfile('./compression/annotation_indexes.bytes',dtype='uint16')
print(np.all(annotation_indexes==annotation_indexes_load))

In [ ]:
# Reconstruct the annotation volume
ai = 0
i = 0
data_load = np.zeros(data.shape)
for ap in np.arange(data.shape[0]):
    for dv in np.arange(data.shape[1]):
        for lr in np.arange(data.shape[2]):
            if volume_indexes_load[i]==1:
                data_load[ap,dv,lr] = annotation_map_load[annotation_indexes_load[ai]]
                ai+=1
            i+=1
print(np.all(data==data_load))